In [1]:
from langdetect import detect
import xml.etree.ElementTree as ET
import json
import requests
import re
import pandas as pd
from pprint import pprint


In [2]:
df = pd.read_csv('clean_songs.csv')
#df =df.head(10)

In [3]:
genres_musicaux = [
    "rock",
    "pop",
    "hip-hop",
    "rap",
    "country",
    "blues",
    "jazz",
    "classical",
    "electronic",
    "dance",
    "reggae",
    "r&b",
    "folk",
    "metal",
    "punk",
    "alternative",
    "indie",
    "soul",
    "funk",
    "disco",
    "gospel",
    "electronic dance music (edm)",
    "techno",
    "house",
    "trance",
    "dubstep",
    "ska",
    "classical",
    "opera",
    "world",
    "k-pop",
    "latin",
    "raggae",
    "heavy metal",
    "singer-songwriter",
    "rap rock",
    "country pop",
    "jazz fusion",
    "electropop",
    "acoustic",
    "pop rock",
    "rap metal",
    "progressive rock",
    "classical crossover",
]

In [4]:
def delete_first_line(chanson):
    chanson = re.sub(r'\[[^\]]*\]', '', chanson)
    lines = chanson.split('\n')
    lyrics = '\n'.join(lines[1:])
    return lyrics

df['Lyrics'] = df['Lyrics'].apply(delete_first_line)
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors
0,10:35,Tiësto,10:35,88,All I know it's 10:35\nAnd I can feel your arm...,23
1,2 Become 1,Spice Girls,Spice,68,Candle light and soul forever\nA dream of you ...,27
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,One is you make me happy\nTwo is you set me fr...,16
3,25 To Life,Eminem,Recovery,64,\nToo late for the other side\nCaught in a cha...,175
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,"Well, I go to work every Monday\nGet to do som...",8
...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,All it takes is a downtown bar\nAll it takes i...,3
483,Your Song,Elton John,Elton John (Remastered Version),0,It's a little bit funny\nThis feeling inside\n...,82
484,You're Gorgeous,Babybird,Ugly Beautiful,56,Y o u r e g o r g e o u s\n\n\nRemember that t...,8
485,You're Makin' Me High,Toni Braxton,Secrets,62,I'll always think of you\nInside of my private...,23


In [5]:
df["lang"] = ""

def detect_lang(row):
    lyrics = row["Lyrics"]
    # Assurez-vous que "Lyrics" est une colonne de type str ou utilisez str(lyrics) pour garantir que c'est une chaîne de caractères
    langue_detectee = detect(lyrics)
    return langue_detectee

# Appliquez la fonction `detect_lang` à chaque ligne du DataFrame
df["Detected_Language"] = df.apply(detect_lang, axis=1)          
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,lang,Detected_Language
0,10:35,Tiësto,10:35,88,All I know it's 10:35\nAnd I can feel your arm...,23,,en
1,2 Become 1,Spice Girls,Spice,68,Candle light and soul forever\nA dream of you ...,27,,en
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,One is you make me happy\nTwo is you set me fr...,16,,en
3,25 To Life,Eminem,Recovery,64,\nToo late for the other side\nCaught in a cha...,175,,en
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,"Well, I go to work every Monday\nGet to do som...",8,,en
...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,All it takes is a downtown bar\nAll it takes i...,3,,en
483,Your Song,Elton John,Elton John (Remastered Version),0,It's a little bit funny\nThis feeling inside\n...,82,,en
484,You're Gorgeous,Babybird,Ugly Beautiful,56,Y o u r e g o r g e o u s\n\n\nRemember that t...,8,,en
485,You're Makin' Me High,Toni Braxton,Secrets,62,I'll always think of you\nInside of my private...,23,,en


In [6]:
def get_lyrics(artist, song_name):
    url = f"http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist={artist}&song={song_name}"

    response = requests.get(url)
    response_text = response.text

    if response.status_code == 200:
        lyric_match = re.search(r'<Lyric>(.*?)</Lyric>', response_text, re.DOTALL)

        if lyric_match:
            lyric_content = lyric_match.group(1)
            return(lyric_content)
        else:
            print("Balise <Lyric> introuvable.")

#print(get_lyrics("lady gaga","girl"))


In [7]:
import requests
import pandas as pd

def song_infos(row):
    artist_name = row["Artist"]
    song_name = row["Name"]
    api_key = 'fbd866cc9dd08da54a1e4ed4f6ff7896'

    url = f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={api_key}&artist={artist_name}&track={song_name}&format=json"

    artist_info_url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    response_artist = requests.get(artist_info_url)

    response = requests.get(url)
    data = response.json()

    artist_data = response_artist.json()

    tags_artist = {}
    tags_song = {}

    # Vérifiez si la clé 'track' est dans la réponse (pour gérer les erreurs)
    if 'track' in data:
        track_data = data['track']
        duration = int(track_data['duration'])/1000
        minutes = duration // 60
        if duration % 60 >= 30:
            minutes += 1  
        
        listener = track_data['listeners']
        playcount = track_data['playcount']

        # Vérifiez si 'wiki' est présent dans les données (pour éviter les erreurs)
        if 'wiki' in track_data:
            published = track_data['wiki'].get('published', 'N/A')
        else:
            published = 'N/A'
        
        
        track_tags = data.get("track", {}).get("toptags", {}).get("tag", [])
        artist_tags = artist_data.get("artist", {}).get("tags", {}).get("tag", [])


        tags_song  = [tag["name"] for tag in track_tags if tag["name"] in genres_musicaux]
        tags_artist = [tag["name"] for tag in artist_tags if tag["name"] in genres_musicaux]
        row['genre_artist']= tags_artist
        row['genre_song']= tags_song
        row['duration']=minutes
        row['listener']=listener
        row['playcount']=playcount
        row['date']=published
    else:
        row['genre_artist']= 'N/A'
        row['genre_song']= 'N/A'
        row['duration']=0
        row['listener']=0
        row['playcount']=0
        row['date']='N/A'
    return row
    


df = df.apply(song_infos, axis=1)

# Concaténez le DataFrame d'origine avec le DataFrame des nouvelles informations

# Affichez le DataFrame mis à jour


In [8]:
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,lang,Detected_Language,genre_artist,genre_song,duration,listener,playcount,date
0,10:35,Tiësto,10:35,88,All I know it's 10:35\nAnd I can feel your arm...,23,,en,"[trance, electronic, dance, techno]",[],3.0,227619,1649557,"08 Nov 2022, 17:18"
1,2 Become 1,Spice Girls,Spice,68,Candle light and soul forever\nA dream of you ...,27,,en,"[pop, dance]",[pop],4.0,306723,1508229,"28 Jul 2009, 01:11"
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,One is you make me happy\nTwo is you set me fr...,16,,en,[electronic],[],3.0,97756,552882,N/A
3,25 To Life,Eminem,Recovery,64,\nToo late for the other side\nCaught in a cha...,175,,en,"[rap, pop]",[rap],4.0,375288,2131369,"21 Jan 2022, 17:36"
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,"Well, I go to work every Monday\nGet to do som...",8,,en,[country],[],4.0,23226,148011,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,All it takes is a downtown bar\nAll it takes i...,3,,en,[country],[country],3.0,21595,123365,N/A
483,Your Song,Elton John,Elton John (Remastered Version),0,It's a little bit funny\nThis feeling inside\n...,82,,en,"[pop, singer-songwriter, rock]",[pop],4.0,1034351,6064549,"06 Sep 2009, 19:16"
484,You're Gorgeous,Babybird,Ugly Beautiful,56,Y o u r e g o r g e o u s\n\n\nRemember that t...,8,,en,"[indie, singer-songwriter]","[pop, indie, alternative]",4.0,91521,401676,N/A
485,You're Makin' Me High,Toni Braxton,Secrets,62,I'll always think of you\nInside of my private...,23,,en,"[soul, pop]",[],4.0,146807,540705,"02 Dec 2008, 00:58"


In [9]:
data2 = {
    'Name': ['New Song 1', 'Groovy Groove', 'Summer Breeze'],
    'Artist': ['Mystery Artist', 'Funky Band', 'Sunny Artist'],
    'Album': ['Secret Album', 'Groove Time', 'Beach Vibes'],
    'Popularity': [50, 72, 65],
    'Lyrics': ['[Verse 1]\nThis is a new song...', '[Chorus]\nDancing to the groovy groove...', '[Verse 1]\nFeeling the summer breeze...'],
    'Nb contributors': [5, 8, 6],
    'lang': ['en', 'en', 'en'],
    'genre_artist': [['pop'], ['rap'], ['pop', 'rap']],
    'genre_song': [['pop'], ['funk', 'rap'], ['pop']],
    'duration': [2.0, 3.0, 4.0],
    'listener': [100000, 80000, 75000],
    'playcount': [5000000, 1000000, 950000],
    'date': [2010.0,2000.0,2000.0]
}

df2 = pd.DataFrame(data2)

import numpy as np

df['date'].replace('N/A', np.nan, inplace=True)

# Convertissez la colonne "date" en datetime en gérant les erreurs avec des valeurs NaT (Not a Timestamp)

df['date'] = pd.to_datetime(df['date'], format='%d %b %Y, %H:%M', errors='coerce')

# Extraction de la décennie, en gérant les valeurs manquantes
df['date'] = (df['date'].dt.year // 10) * 10

df



,Name,Artist,Album,Popularity,Lyrics,Nb contributors,lang,Detected_Language,genre_artist,genre_song,duration,listener,playcount,date
0,10:35,Tiësto,10:35,88,All I know it's 10:35\nAnd I can feel your arm...,23,,en,"[trance, electronic, dance, techno]",[],3.0,227619,1649557,2020.0
1,2 Become 1,Spice Girls,Spice,68,Candle light and soul forever\nA dream of you ...,27,,en,"[pop, dance]",[pop],4.0,306723,1508229,2000.0
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,One is you make me happy\nTwo is you set me fr...,16,,en,[electronic],[],3.0,97756,552882,NaN
3,25 To Life,Eminem,Recovery,64,\nToo late for the other side\nCaught in a cha...,175,,en,"[rap, pop]",[rap],4.0,375288,2131369,2020.0
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,"Well, I go to work every Monday\nGet to do som...",8,,en,[country],[],4.0,23226,148011,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,All it takes is a downtown bar\nAll it takes i...,3,,en,[country],[country],3.0,21595,123365,NaN
483,Your Song,Elton John,Elton John (Remastered Version),0,It's a little bit funny\nThis feeling inside\n...,82,,en,"[pop, singer-songwriter, rock]",[pop],4.0,1034351,6064549,2000.0
484,You're Gorgeous,Babybird,Ugly Beautiful,56,Y o u r e g o r g e o u s\n\n\nRemember that t...,8,,en,"[indie, singer-songwriter]","[pop, indie, alternative]",4.0,91521,401676,NaN
485,You're Makin' Me High,Toni Braxton,Secrets,62,I'll always think of you\nInside of my private...,23,,en,"[soul, pop]",[],4.0,146807,540705,2000.0


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             3 non-null      object 
 1   Artist           3 non-null      object 
 2   Album            3 non-null      object 
 3   Popularity       3 non-null      int64  
 4   Lyrics           3 non-null      object 
 5   Nb contributors  3 non-null      int64  
 6   lang             3 non-null      object 
 7   genre_artist     3 non-null      object 
 8   genre_song       3 non-null      object 
 9   duration         3 non-null      float64
 10  listener         3 non-null      int64  
 11  playcount        3 non-null      int64  
 12  date             3 non-null      float64
dtypes: float64(2), int64(4), object(7)
memory usage: 444.0+ bytes


In [11]:
import pandas as pd
df['points'] = 0

def increment_points(df, column_name, value):
    df['points'] += df[column_name].apply(lambda x: 1 if x == value else 0)


for song in df2["Name"]:
    increment_points(df, 'Name', song)

for artist in df2["Artist"]:
    increment_points(df, 'Artist', artist)

for album in df2["Album"]:
    increment_points(df, 'Album', album)

for popularity in df2["Popularity"]:
    increment_points(df, 'Popularity', popularity)

for contributor in df2["Nb contributors"]:
    increment_points(df, 'Nb contributors', contributor)

for lang in df2["lang"]:
    increment_points(df, 'lang', lang)

print(df['points'])
def calculate_similarity(row, column_name):
    points = 0
    for genre_list in df2[column_name]:
        points += sum(1 for genre in row[column_name] if genre in genre_list)
    return points

df['points'] += df.apply(calculate_similarity, args=('genre_artist',), axis=1)

# Calculate and update the 'points' column for 'genre_song'
df['points'] += df.apply(calculate_similarity, args=('genre_song',), axis=1)

print(df['points'])
for duration in df2["duration"]:
    increment_points(df, 'duration', duration)

for listener in df2["listener"]:
    increment_points(df, 'listener', listener)

for playcount in df2["playcount"]:
    increment_points(df, 'playcount', playcount)

for date in df2["date"]:
    increment_points(df, 'date', date)

print(df)
df

0      0
1      0
2      0
3      0
4      1
      ..
482    0
483    0
484    1
485    0
486    0
Name: points, Length: 487, dtype: int64
0      0
1      4
2      0
3      5
4      1
      ..
482    0
483    4
484    3
485    2
486    0
Name: points, Length: 487, dtype: int64
                                  Name        Artist  \
0                                10:35        Tiësto   
1                           2 Become 1   Spice Girls   
2    21 Reasons (feat. Ella Henderson)   Nathan Dawe   
3                           25 To Life        Eminem   
4                        5 Leaf Clover    Luke Combs   
..                                 ...           ...   
482                 Your Heart Or Mine     Jon Pardi   
483                          Your Song    Elton John   
484                    You're Gorgeous      Babybird   
485              You're Makin' Me High  Toni Braxton   
486                               アイドル       YOASOBI   

                                 Album  Popularit

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,lang,Detected_Language,genre_artist,genre_song,duration,listener,playcount,date,points
0,10:35,Tiësto,10:35,88,All I know it's 10:35\nAnd I can feel your arm...,23,,en,"[trance, electronic, dance, techno]",[],3.0,227619,1649557,2020.0,1
1,2 Become 1,Spice Girls,Spice,68,Candle light and soul forever\nA dream of you ...,27,,en,"[pop, dance]",[pop],4.0,306723,1508229,2000.0,7
2,21 Reasons (feat. Ella Henderson),Nathan Dawe,21 Reasons (feat. Ella Henderson),81,One is you make me happy\nTwo is you set me fr...,16,,en,[electronic],[],3.0,97756,552882,NaN,1
3,25 To Life,Eminem,Recovery,64,\nToo late for the other side\nCaught in a cha...,175,,en,"[rap, pop]",[rap],4.0,375288,2131369,2020.0,6
4,5 Leaf Clover,Luke Combs,Gettin' Old,75,"Well, I go to work every Monday\nGet to do som...",8,,en,[country],[],4.0,23226,148011,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Your Heart Or Mine,Jon Pardi,Mr. Saturday Night,76,All it takes is a downtown bar\nAll it takes i...,3,,en,[country],[country],3.0,21595,123365,NaN,1
483,Your Song,Elton John,Elton John (Remastered Version),0,It's a little bit funny\nThis feeling inside\n...,82,,en,"[pop, singer-songwriter, rock]",[pop],4.0,1034351,6064549,2000.0,7
484,You're Gorgeous,Babybird,Ugly Beautiful,56,Y o u r e g o r g e o u s\n\n\nRemember that t...,8,,en,"[indie, singer-songwriter]","[pop, indie, alternative]",4.0,91521,401676,NaN,4
485,You're Makin' Me High,Toni Braxton,Secrets,62,I'll always think of you\nInside of my private...,23,,en,"[soul, pop]",[],4.0,146807,540705,2000.0,5


In [12]:
df = df.sort_values(by='points', ascending=False).reset_index(drop=True)
df

,Name,Artist,Album,Popularity,Lyrics,Nb contributors,lang,Detected_Language,genre_artist,genre_song,duration,listener,playcount,date,points
0,Woman,Doja Cat,Planet Her,56,"Hey, woman\nHey, woman\n\n\nWoman\nLet me be y...",163,,en,"[pop, rap]","[rap, pop]",3.0,976609,14862608,2020.0,8
1,In Your Head,Eminem,Revival,64,"What's in your head, in your head\nZombie, zom...",169,,en,"[rap, pop]",[pop],3.0,74979,403888,2010.0,8
2,Without Me,Eminem,The Eminem Show,90,"Obie Trice, real name, no gimmicks *record scr...",439,,en,"[rap, pop]",[rap],4.0,1489883,11452904,2000.0,8
3,We Made You,Eminem,Relapse: Refill,67,"Guess who? D'you miss me?\nJessica Simpson, si...",210,,en,"[rap, pop]",[rap],4.0,487243,2538551,2000.0,8
4,Beautiful,Eminem,Relapse: Refill,72,Lately I've been hard to reach\nI've been too ...,289,,en,"[rap, pop]",[rap],7.0,465453,3725366,2000.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,What Is Real?,The Band That Saved The World,Name in Lights,33,"\nI go there nearly every night, I find the ac...",1,,en,[],[],6.0,105,168,NaN,0
483,The Family Madrigal,Stephanie Beatriz,Encanto (Original Motion Picture Soundtrack),73,Drawers!\nFloors!\nDoors!\nLet's go!\n\n\nThis...,85,,en,[],[],0.0,138566,1025274,NaN,0
484,Where Are You Now,Lost Frequencies,Where Are You Now,87,You're just like my favorite song going 'round...,18,,en,"[electronic, dance]","[dance, electronic]",5.0,300393,2379415,2020.0,0
485,TMO (Turn Me On) [feat. Kevin Lyttle],Luude,TMO (Turn Me On) [feat. Kevin Lyttle],56,"Oh yeah\nYeah, yes\nStyle up from Luude, 'long...",3,,en,[electronic],[],0.0,18948,84419,NaN,0
